<a href="https://colab.research.google.com/github/Pestrdan/netology_research/blob/main/Deep_learning_(Pytorch)/DLL_35_HW6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd  # для работы с данными
import time  # для оценки времени
import torch  # для написания нейросети

## Загрузка данных

Будем работать с датасетом реплик из Симпсонов. Нам нужно извлечь предобработанные тексты и закодировать их числами

In [ ]:
df = pd.read_csv('simpsons_script_lines.csv')
#df.head()

<ipython-input-4-40135c10baa1>:1: DtypeWarning: Columns (4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('simpsons_script_lines.csv')


In [ ]:
#df.info()

In [ ]:
phrases = df['normalized_text'].tolist()  # колонка с предобработанными текстами
#phrases[:10]

In [ ]:
text = [[c for c in ph] for ph in phrases if type(ph) is str]

# **Задание 1**

## Создаём массив с данными

Нужно

1. Разбить данные на токены (у нас символы)
2. Закодировать числами
3. Превратить в эмбеддинги

In [ ]:
CHARS = set('abcdefghijklmnopqrstuvwxyz')
INDEX_TO_CHAR_PRE = [w for w in CHARS]
INDEX_TO_CHAR_PRE.sort()
INDEX_TO_CHAR = ['none'] + [' '] + INDEX_TO_CHAR_PRE
CHAR_TO_INDEX_Y = {w: i for i, w in enumerate(INDEX_TO_CHAR)}
#CHAR_TO_INDEX

In [ ]:
step = 3
CHAR_TO_INDEX_X = {}
for key, value in CHAR_TO_INDEX_Y.items():
  CHAR_TO_INDEX_X[key] = value + step
#CHAR_TO_INDEX_X

In [ ]:
MAX_LEN = 50  # мы хотим ограничить максимальную длину ввода
X = torch.zeros((len(text), MAX_LEN), dtype=int)  # создаём пустой вектор для текста, чтобы класть в него индексы токенов
for i in range(len(text)):  # для каждого предложения
    for j, w in enumerate(text[i]):  # для каждого токена
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX_X.get(w, CHAR_TO_INDEX_X['none'])

In [ ]:
MAX_LEN = 50  # мы хотим ограничить максимальную длину ввода
Y = torch.zeros((len(text), MAX_LEN), dtype=int)  # создаём пустой вектор для текста, чтобы класть в него индексы токенов
for i in range(len(text)):  # для каждого предложения
    for j, w in enumerate(text[i]):  # для каждого токена
        if j >= MAX_LEN:
            break
        Y[i, j] = CHAR_TO_INDEX_Y.get(w, CHAR_TO_INDEX_Y['none'])

In [ ]:
X[0]

tensor([18, 19,  4,  5,  7, 24, 25,  5, 16, 16, 29,  4, 13, 24,  4, 27,  5, 23,
         4,  5,  4, 16, 13, 24, 24, 16,  9,  4, 19, 10,  4,  6, 19, 24, 12,  4,
        23, 19, 17,  9, 24, 13, 17,  9, 23,  4, 27, 12,  9, 18])

In [ ]:
Y[0]

tensor([15, 16,  1,  2,  4, 21, 22,  2, 13, 13, 26,  1, 10, 21,  1, 24,  2, 20,
         1,  2,  1, 13, 10, 21, 21, 13,  6,  1, 16,  7,  1,  3, 16, 21,  9,  1,
        20, 16, 14,  6, 21, 10, 14,  6, 20,  1, 24,  9,  6, 15])

In [ ]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.embedding = torch.nn.Embedding(len(CHAR_TO_INDEX_X) + step, len(CHAR_TO_INDEX_X))
        self.rnn = torch.nn.RNN(len(CHAR_TO_INDEX_X), 128)
        self.out = torch.nn.Linear(128, len(CHAR_TO_INDEX_X) + step)

    def forward(self, sentences, state=None):
        x = self.embedding(sentences)
        x, s = self.rnn(x) # берём выход с последнего слоя для всех токенов, а не скрытое состояние
        return self.out(x)

In [ ]:
model = Network()

In [ ]:
criterion = torch.nn.CrossEntropyLoss()  # типичный лосс многоклассовой классификации
optimizer = torch.optim.SGD(model.parameters(), lr=.01)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
dataset = TensorDataset(X, Y)

In [ ]:
BATCH_SIZE = 100

In [ ]:
iter = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE)

In [ ]:
for ep in range(5):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for x_in, y_in in iter:
        x_in = x_in
        y_in = y_in.view(1, -1).squeeze()
        optimizer.zero_grad()
        y_pred = model.forward(x_in).view(-1, len(INDEX_TO_CHAR) + step)

        loss = criterion(y_pred, y_in)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))

Epoch 0. Time: 27.186, Train loss: 0.812
Epoch 1. Time: 26.916, Train loss: 0.131
Epoch 2. Time: 26.951, Train loss: 0.057
Epoch 3. Time: 26.917, Train loss: 0.036
Epoch 4. Time: 26.313, Train loss: 0.027


In [ ]:
def generate_sentence(word):
    sentence = list(word)
    sentence = [CHAR_TO_INDEX_Y.get(s, 0) for s in sentence]
    answers = model.forward(torch.tensor(sentence))
    probas, indices = answers.topk(1)
    return ''.join([INDEX_TO_CHAR[ind.item()] for ind in indices.flatten()])

In [ ]:
generate_sentence('khoorczruog')

'hello world'

# **Задание 2**

In [ ]:
import random

In [ ]:
new_text = ''
for i in phrases:
  new_text += str(i)
  new_text += ' '

In [ ]:
import re

In [ ]:
new_text = re.sub('[^a-z ]', ' ', new_text)
new_text = re.sub('\s+', ' ', new_text)

In [ ]:
INDEX_TO_CHAR = sorted(list(set(new_text)))
CHAR_TO_INDEX = {c: i for i, c in enumerate(INDEX_TO_CHAR)}

In [ ]:
CHAR_TO_INDEX

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [ ]:
MAX_LEN = 40
STEP = 3
SENTENCES = []
NEXT_CHARS = []
for i in range(0, len(new_text) - MAX_LEN, STEP):
    SENTENCES.append(new_text[i: i + MAX_LEN])
    NEXT_CHARS.append(new_text[i + MAX_LEN])
print('Num sents:', len(SENTENCES))

Num sents: 2253060


In [ ]:
print('Vectorization...')
X = torch.zeros((len(SENTENCES), MAX_LEN), dtype=int)
Y = torch.zeros((len(SENTENCES)), dtype=int)
for i, sentence in enumerate(SENTENCES):
    for t, char in enumerate(sentence):
        X[i, t] = CHAR_TO_INDEX[char]
    Y[i] = CHAR_TO_INDEX[NEXT_CHARS[i]]

Vectorization...


In [ ]:
BATCH_SIZE=512
dataset = TensorDataset(X, Y)
data = DataLoader(dataset, BATCH_SIZE, shuffle=True)

In [ ]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, dictionary_size, embedding_size, num_hiddens, num_classes):
        super().__init__()

        self.num_hiddens = num_hiddens
        self.embedding = torch.nn.Embedding(dictionary_size, embedding_size)
        self.hidden = torch.nn.RNN(embedding_size, num_hiddens, batch_first=True)
        self.output = torch.nn.Linear(num_hiddens, num_classes)

    def forward(self, X):
        out = self.embedding(X)
        _, state = self.hidden(out)
        predictions = self.output(state[0].squeeze())
        return predictions

In [ ]:
def sample(preds):
    softmaxed = torch.softmax(preds, 0)
    probas = torch.distributions.multinomial.Multinomial(1, softmaxed).sample()
    return probas.argmax()

In [ ]:
def generate_text():
    start_index = random.randint(0, len(new_text) - MAX_LEN - 1)

    generated = ''
    sentence = new_text[start_index: start_index + MAX_LEN]
    generated += sentence

    for i in range(MAX_LEN):
        x_pred = torch.zeros((1, MAX_LEN), dtype=int)
        for t, char in enumerate(generated[-MAX_LEN:]):
            x_pred[0, t] = CHAR_TO_INDEX[char]

        preds = model(x_pred.cuda()).cpu()
        next_char = INDEX_TO_CHAR[sample(preds)]
        generated = generated + next_char

    print(generated[:MAX_LEN] + '|' + generated[MAX_LEN:])

In [ ]:
model = NeuralNetwork(len(CHAR_TO_INDEX), 64, 128, len(CHAR_TO_INDEX))

In [ ]:
model = model.cuda()

In [ ]:
criterion = torch.nn.CrossEntropyLoss()  # типичный лосс многоклассовой классификации
optimizer = torch.optim.SGD(model.parameters(), lr=.01)

In [ ]:
#scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.1)

In [ ]:
for ep in range(20):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    model.train()
    for X_b, y_b in data:
        X_b, y_b = X_b.cuda(), y_b.cuda()
        optimizer.zero_grad()
        answers = model(X_b)
        loss = criterion(answers, y_b)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    #scheduler.step()

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
    model.eval()
    generate_text()

Epoch 0. Time: 28.292, Train loss: 2.302
out boy its like looking into a living s|ouf thes in whow nis moner goop ontesfer
Epoch 1. Time: 28.796, Train loss: 2.062
eling oh man hell blow our winning strea|s the wive wise pages stontn nanles i al
Epoch 2. Time: 28.371, Train loss: 1.970
 gift what could a girl have that i nan |nant naw gond and just in and the read c
Epoch 3. Time: 28.345, Train loss: 1.909
 witnesses come in arthur now where did |entoutsers goo hirgigel coed mksivew mor
Epoch 4. Time: 28.164, Train loss: 1.865
your age im sure you understand things c|reatinf ficking and wouk the if fry hanc
Epoch 5. Time: 27.967, Train loss: 1.830
 slack well how many should i get well w|e who pack who helcoute this i were olm 
Epoch 6. Time: 28.494, Train loss: 1.802
ove at all we call those weekdays good o|f year pelpets fonits moithing becto bar
Epoch 7. Time: 28.574, Train loss: 1.779
 looks the same it levels the playing fi|red mater up inach up look perolighte hi
Epoch 8. Time: 2